In [1]:
# I chose the Exponential Smoothing (Holt-Winters) model

from pathlib import Path
from typing import Union
import pickle
import pandas as pd
from statsmodels.tsa.holtwinters import ExponentialSmoothing


def _prep_train_series(train_df: pd.DataFrame) -> pd.Series:
    """
    Prepare the training target as an hourly-frequency Series indexed by Timestamp.
    Ensures datetime index, sorting, numeric coercion, hourly freq, and fills rare gaps.
    """
    df = train_df.copy()
    df["Timestamp"] = pd.to_datetime(df["Timestamp"])
    df = df.sort_values("Timestamp").set_index("Timestamp")

    y = pd.to_numeric(df["trips"], errors="coerce")
    # Force hourly frequency to keep Holt-Winters stable across environments
    y = y.asfreq("H")

    # Fill any occasional missing values to maintain a clean hourly series
    if y.isna().any():
        y = y.interpolate(limit_direction="both")

    return y


def _prep_test_index(test_df: pd.DataFrame) -> pd.DatetimeIndex:
    """
    Return a DatetimeIndex for test timestamps, preserving hourly freq when possible.
    """
    idx = pd.to_datetime(test_df["Timestamp"])
    try:
        idx = pd.DatetimeIndex(idx).asfreq("H")
    except Exception:
        idx = pd.DatetimeIndex(idx)
    return idx


def model1(train_df: pd.DataFrame):
    """
    Fit an Exponential Smoothing (Holt-Winters) model with additive trend and
    additive daily seasonality (24 hours). Returns a fitted statsmodels object.
    Assumes train_df has columns ['Timestamp', 'trips'].
    """
    y = _prep_train_series(train_df)

    # Build model with backward-compatible init (older statsmodels may not
    # support initialization_method). Also enable brute-force if available.
    try:
        model = ExponentialSmoothing(
            y, trend="add", seasonal="add", seasonal_periods=24,
            initialization_method="estimated"
        )
        fitted = model.fit(optimized=True, use_brute=True)
    except TypeError:
        # Fallback for older statsmodels signatures
        model = ExponentialSmoothing(
            y, trend="add", seasonal="add", seasonal_periods=24
        )
        fitted = model.fit(optimized=True, use_brute=True)

    return fitted


def predict(fitted_model, test_df: pd.DataFrame) -> pd.Series:
    """
    Forecast len(test_df) steps ahead and return a pd.Series named 'pred'
    indexed by the test timestamps.
    """
    h = len(test_df)
    fc = fitted_model.forecast(steps=h)
    idx = _prep_test_index(test_df)
    return pd.Series(fc.values, index=idx, name="pred")


def save_model(fitted_model, path: Union[str, Path] = "model1.pkl") -> Path:
    """
    Persist a fitted model to disk using pickle.
    """
    p = Path(path)
    with p.open("wb") as f:
        pickle.dump(fitted_model, f)
    return p


def load_model(path: Union[str, Path] = "model1.pkl"):
    """
    Load a previously saved fitted model from disk.
    """
    p = Path(path)
    with p.open("rb") as f:
        return pickle.load(f)
